In [ ]:
import rasterio
import matplotlib.pyplot as plt
import warnings
import numpy as np
import pandas as pd
import os
import gdal
from sklearn.metrics import mean_squared_error
from math import sqrt

# Loading Images

In [ ]:
ref_path = "20180819T090551.tif"
sub_path = "winter_20190106T091351.tif"
ref_im = rasterio.open(ref_path).read()
sub_im = rasterio.open(sub_path).read()

ref_im = ref_im.astype(float)
sub_im = sub_im.astype(float)

print(ref_path,ref_im.dtype)
print(sub_path,sub_im.dtype)

bands = ref_im.shape[0]
x_dim = ref_im.shape[1]
y_dim = ref_im.shape[2]

# Image Normalization

In [ ]:
def normalize(array):
  # return (array - array.min()) / (array.max() - array.min())
  return (array) / (array.max())

def hist_norm(array):
    array_min, array_max = np.percentile(array, [1, 99])
    return (array - array_min) / (array_max - array_min)

for i in range(bands):
  temp_ref = ref_im[i,:,:]/10000
  temp_ref[temp_ref>1] = 1.0
  temp_sub = sub_im[i,:,:]/10000
  temp_sub[temp_sub>1] = 1.0
  ref_im[i,:,:] = temp_ref
  sub_im[i,:,:] = temp_sub
    
  print("--Band ", i+1," statistics--")
  print("Ref Min: ", ref_im[i].min())
  print("Ref Max: ", ref_im[i].max())
  print("Ref Type: ", ref_im.dtype)
  print("Ref Std: ", np.std(ref_im[i]),"\n")
  print("Sub Min: ", sub_im[i].min())
  print("Sub Max: ", sub_im[i].max())
  print("Sub Type: ", sub_im.dtype)
  print("Sub Std: ", np.std(sub_im[i]),"\n")
  

In [ ]:
f, axarr = plt.subplots(1,2,figsize=(30,30))
# axarr[0].imshow(np.dstack((hist_norm(ref_im[2]),hist_norm(ref_im[1]),hist_norm(ref_im[0]))))
# axarr[1].imshow(np.dstack((hist_norm(sub_im[2]),hist_norm(sub_im[1]),hist_norm(sub_im[0]))))
axarr[0].imshow(np.dstack((ref_im[2],ref_im[1],ref_im[0])))
axarr[1].imshow(np.dstack((sub_im[2],sub_im[1],sub_im[0])))

# Calculating AND and ANRD

In [ ]:
k = 2
AND= np.zeros((bands,x_dim,y_dim))
ANRD = np.zeros((bands, x_dim,y_dim))
for j in range(bands):
  for x in range(x_dim):
    for y in range(y_dim):
      refj = ref_im[j,x,y]
      subj = sub_im[j,x,y]
      refk = ref_im[k,x,y]
      subk = sub_im[k,x,y]
      AND[j,x,y] = abs(refj-subj) / abs(refj+subj)
      ANRD[j,x,y] = abs( (refj/refk) - (subj/subk) ) / abs( (refj/refk) + (subj/subk) )
  print("Band ", j+1)
  print("AND std: ", np.std(AND[j]))
  print("ANRD std: ", np.std(ANRD[j]),"\n")

plt.figure(figsize=(20,20))
f, axarr = plt.subplots(4,2,figsize=(30,30))
for i in range(bands):
    axarr[i,0].imshow(AND[i],cmap='gray')
    axarr[i,1].imshow(ANRD[i],cmap='gray')
plt.show()

# Calculating optimal n

In [ ]:
AND_threshold = np.empty(bands)
ANRD_threshold = np.empty(bands)
optimal_n = np.empty(bands)
for i in range(bands):
    n = 1.0
    error = 10
    count_list = []
    n_list = []
    while abs(error) > 1e-1:
      AND_threshold[i] = np.linalg.norm(np.min(AND[i]) + np.std(AND[i])/n)
      error = AND_threshold[i] - np.linalg.norm(np.min(AND[i]))
      count_list.append(len(AND[i][AND[i] <= AND_threshold[i]]))
      n_list.append(n)
      n += 0.05
    #   print("-- Error=%.3f  n=%.1f  count=%d --"%(error,n,len(AND[AND <= AND_threshold])))

    #optimal n
    with warnings.catch_warnings():
      warnings.simplefilter('ignore', np.RankWarning)
      log_fit = np.polyfit(count_list, n_list,20) 
      p = np.poly1d(log_fit)
    mean_count = np.average(count_list) # mean count
    optimal_n[i] = p(mean_count) 
    AND_threshold[i] = np.linalg.norm(np.min(AND[i]) + np.std(AND[i])/optimal_n[i])
    ANRD_threshold[i] = np.linalg.norm(np.min(ANRD[i]) + np.std(ANRD[i])/optimal_n[i])
    if i == 3:
        ANRD_threshold[k] = ANRD_threshold[k-1]
    with plt.style.context('dark_background'):
      plt.figure(figsize=(6,6))
      plt.plot(count_list,n_list,'o',color='orange')
      plt.plot(count_list,p(count_list))
      plt.xlabel("Pixel count")
      plt.ylabel("n")
    print("-- Band ", i+1," --")
    print("AND threshold value: ",AND_threshold[i]," ANRD threshold value: ",ANRD_threshold[i]," Optimal n value : ",optimal_n[i])

# Finding Invariant Pixels for each band using the thresholds

In [ ]:
IPx = []
IPy = []
IPref = []
IPsub = []
IP_im = np.zeros((bands,x_dim,y_dim))
for b in range(bands):
  tempx = []
  tempy = []
  temp_ref = []
  temp_sub =[]
  for x in range(1,x_dim-1):
    for y in range(1,y_dim-1):
      if (AND[b,x,y] < AND_threshold[b]) and (ANRD[b,x,y] < ANRD_threshold[b]):
        if (AND[b,x-1,y-1] < AND_threshold[b]) and(AND[b,x+1,y+1] < AND_threshold[b]):
          tempx.append(x)
          tempy.append(y)
          temp_ref.append(ref_im[b,x,y])
          temp_sub.append(sub_im[b,x,y])
          IP_im[b,x,y] = 1
  IPx.append(tempx)
  IPy.append(tempy)
  IPref.append(temp_ref)
  IPsub.append(temp_sub)

f, axarr = plt.subplots(2,2,figsize=(30,23))
axarr[0,0].imshow(IP_im[0],cmap='gray')
axarr[0,1].imshow(IP_im[1],cmap='gray')
axarr[1,0].imshow(IP_im[2],cmap='gray')
axarr[1,1].imshow(IP_im[3],cmap='gray')

In [ ]:
#Dataframes for each band
blue = pd.DataFrame({
    "IPx": IPx[0],
    "IPy": IPy[0],
    "Reference Image Value": IPref[0],
    "Subject Image Value": IPsub[0]
})

green = pd.DataFrame({
    "IPx": IPx[1],
    "IPy": IPy[1],
    "Reference Image Value": IPref[1],
    "Subject Image Value": IPsub[1]
})

red = pd.DataFrame({
    "IPx": IPx[2],
    "IPy": IPy[2],
    "Reference Image Value": IPref[2],
    "Subject Image Value": IPsub[2]
})

nir = pd.DataFrame({
    "IPx": IPx[3],
    "IPy": IPy[3],
    "Reference Image Value": IPref[3],
    "Subject Image Value": IPsub[3]
})
df = pd.DataFrame({
      "Bands": [blue,green,red,nir]
    })
print(df)
df["Bands"][0]
# df["Bands"][0], df["Bands"][1], df["Bands"][2], df["Bands"][3]

# Regression and DRP implementation

In [ ]:
cb = 0
for i in IPx:
  cb = cb + 1
  print('Number of IPs in Band ',cb,' :',len(i))

p_im = []
lines_to_drop = []
for i in range(bands):
  im_fit = np.polyfit(df["Bands"][i]["Subject Image Value"],
                      df["Bands"][i]["Reference Image Value"],
                      1)
  p_im.append(np.poly1d(im_fit))
  print(im_fit)
  pred_sub = p_im[i](df["Bands"][i]["Subject Image Value"])
  df["Bands"][i]["Predicted"] = pred_sub
  std = np.std(pred_sub)
  temp = []
  for j in range(len(pred_sub)):
    DRP = abs(df["Bands"][i]["Subject Image Value"][j] - df["Bands"][i]["Predicted"][j])
    if DRP > 2*std:
      temp.append(j)
  lines_to_drop.append(temp)

for b in range(bands):
    df["Bands"][b] = df["Bands"][b].drop(df["Bands"][b].index[lines_to_drop[b]])
    print('IPs dropped for Band ',b+1,' : ',len(lines_to_drop[b]))

p_im = []
for b in range(bands):
  im_fit = np.polyfit(df["Bands"][b]["Subject Image Value"],
                      df["Bands"][b]["Reference Image Value"],
                      1)
  p_im.append(np.poly1d(im_fit))
  print(im_fit)
  pred_sub = p_im[b](df["Bands"][b]["Subject Image Value"])
  df["Bands"][b]["After_DRP"] = pred_sub
  print("Final number of IPs in Band",b+1,":",df["Bands"][b]["After_DRP"].shape[0])

with plt.style.context('dark_background'):
  plt.figure(figsize=(15,15))
  for b in range(bands):
    ax = plt.subplot(2, 2, b+1)
    ax.plot(df["Bands"][b]["Subject Image Value"],df["Bands"][b]["Reference Image Value"],'o',color='orange',markersize=0.5)
    ax.plot(df["Bands"][b]["Subject Image Value"],df["Bands"][b]["After_DRP"])
    plt.xlabel("Subject Values")
    plt.ylabel("Reference Values")

In [ ]:
# for i in range(bands):
#   print("--Band ", i+1," statistics--")
#   print("Pred Min: ", df["Bands"][i]["Predicted"].min())
#   print("Pred Max: ", df["Bands"][i]["Predicted"].max())
#   print("Pred Std: ", np.std(df["Bands"][i]["Predicted"]))
#   print("After DRP Min: ", df["Bands"][i]["After_DRP"].min())
#   print("After DRP Max: ", df["Bands"][i]["After_DRP"].max())
#   print("After DRP Std: ", np.std(df["Bands"][i]["After_DRP"]),"\n")

# Calculating RMSE

In [ ]:
_temp = []
for i in range(bands):
    b = p_im[i][0]
    a = p_im[i][1]
    temp_sub = (ref_im[i]/a)-b
    new_ref = a*temp_sub-b
    _temp.append(sqrt(mean_squared_error(ref_im[i], new_ref))/2)

df["RMSE"] = _temp
df

# Constructing new subject image

In [ ]:
new_sub_im = np.empty(sub_im.shape)
new_norm_sub_im = np.empty(sub_im.shape)
for b in range(bands):
  new_sub_im[b] = p_im[b](sub_im[b])
  print("--Band ", b+1," statistics--")
  print("Min: ", new_sub_im[b].min())
  print("Max: ", new_sub_im[b].max())
  print("Type: ", new_sub_im.dtype)
  print("Std: ", np.std(new_sub_im[b]),"\n")
#   print("NEW")
#   print(new_sub_im[b])
#   print("NORMED")
#   print(new_norm_sub_im[b])
#   print("OLD")
#   print(sub_im[b])

In [ ]:
r,g,b = (2,1,0)
f, axarr = plt.subplots(1,2,figsize=(30,30))
axarr[0].imshow(np.dstack((sub_im[r],sub_im[g],sub_im[b])))
axarr[1].imshow(np.dstack((new_sub_im[r],new_sub_im[g],new_sub_im[b])))
# axarr[0].imshow(np.dstack((hist_norm(sub_im[r]),hist_norm(sub_im[g]),hist_norm(sub_im[b]))))
# axarr[1].imshow(np.dstack((hist_norm(new_sub_im[r]),hist_norm(new_sub_im[g]),hist_norm(new_sub_im[b]))))

In [ ]:
#@title Displaying original and result for each band
# f, axarr = plt.subplots(4,2,figsize=(30,30))
# for i in range(sub_im.shape[0]):
#   axarr[i,0].imshow(sub_im[i],cmap='gray')
#   axarr[i,1].imshow(new_sub_im[i],cmap='gray')

# Exporting new subject image

In [ ]:
data0 = gdal.Open("20180615T090549.tif")
geotrans=data0.GetGeoTransform()
proj=data0.GetProjection()
temp = np.uint16(np.round(new_sub_im[i]*10000))
dst_filename = 'Exports\\'+sub_path[:-4] + '--RSE--'+ref_path[:-4]+'.tiff'
x_pixels = new_sub_im.shape[2]  # number of pixels in x
y_pixels = new_sub_im.shape[1]  # number of pixels in y
driver = gdal.GetDriverByName('GTiff')
dataset = driver.Create(dst_filename,x_pixels, y_pixels, 4, gdal.GDT_UInt16) # GDT_Float32 GDT_UInt16
dataset.GetRasterBand(1).WriteArray(np.uint16(np.round(new_sub_im[0]*10000)))
dataset.GetRasterBand(2).WriteArray(np.uint16(np.round(new_sub_im[1]*10000)))
dataset.GetRasterBand(3).WriteArray(np.uint16(np.round(new_sub_im[2]*10000)))
dataset.GetRasterBand(4).WriteArray(np.uint16(np.round(new_sub_im[3]*10000)))
dataset.SetGeoTransform(geotrans)
dataset.SetProjection(proj)
dataset.FlushCache()
dataset=None
for i in range(bands):
    temp = np.uint16(np.round(new_sub_im[i]*10000))
    print("--Band ", i+1," statistics--")
    print("Min: ", temp.min())
    print("Max: ", temp.max())
    print("Type: ", temp.dtype)
    print("Std: ", np.std(temp),"\n")
# im = gdal.Open("new_20190116T091321--1-nonorm.tiff",gdal.GDT_UInt16).ReadAsArray()
# print(im.max())
# np.allclose(im,temp)

In [ ]:
# plt.figure()
# plt.imshow(im-np.round(new_sub_im[0]*10000))
# np.allclose(im,np.round(new_sub_im[0]*10000))

In [ ]:
gdal.VersionInfo()

In [ ]:
plt.hist(new_sub_im[0].ravel())

In [ ]:
plt.hist(sub_im[0].ravel())